# Calculating and comparing crop mask areas in Senegal

In [1]:
import pandas as pd
from pathlib import Path
import rasterio
import rasterio.mask
from rasterio.plot import show_hist
from rasterio.plot import show

import rasterstats
from descartes import PolygonPatch

from matplotlib import pyplot as plt
import geojson
from shapely.geometry import Point, mapping, shape
from fiona import collection
import fiona
import geopandas as gpd
import geoplot

Contributor: Susanne Brockmann

## Setup data access

The dataset "Agriculture_yield_area_production_Senegal.xlsm - departments_yields.csv" is a csv download of the following google sheet: https://docs.google.com/spreadsheets/d/11g3BjY398T8jLUbrTi46ac0jbxWgiE-O/edit#gid=2065961712


In [2]:
shape_file_folder = Path.cwd().parent.parent.joinpath('data', 'GADM', 'Senegal')
#print(shape_file_folder)
shape_file_name = "gadm36_SEN_2.shp"
shape_file_path = Path(shape_file_folder).joinpath(shape_file_name)
print(shape_file_path)

H:\Omdena-food\GPSDD-Food-Security-Senegal\data\GADM\Senegal\gadm36_SEN_2.shp


In [3]:
data_folder = Path.cwd().parent.parent.joinpath('data', )
#print(data_folder)
file_name = "Agriculture_yield_area_production_Senegal.xlsm - departments_yields.csv"
input_file_path = Path(data_folder).joinpath("external", file_name)
print(input_file_path)

H:\Omdena-food\GPSDD-Food-Security-Senegal\data\external\Agriculture_yield_area_production_Senegal.xlsm - departments_yields.csv


In [4]:
crop_yield = pd.read_csv(input_file_path)

In [5]:
crop_yield.head()

,Region,Department,Year,Crop,Area planted (ha),Production (T),Yield (ton/ha),Total surface area department (ha),percentage of area planted in region for crop type,Exact locations?,data from http://www.maer.gouv.sn/focus/statistiques-cles/
0,DAKAR,Rufisque,2014/2015,Arachide,2943.0,589,0.200136,130000,2.263846,NaN,NaN
1,DAKAR,Rufisque,2015/2016,Arachide,1233.0,935,0.758313,130000,0.948462,NaN,NaN
2,DAKAR,Rufisque,2016/2017,Arachide,1248.0,257,0.205929,130000,0.960000,NaN,NaN
3,DAKAR,Rufisque,2014/2015,Cowpea,1178.0,177,0.150255,130000,0.906154,NaN,NaN
4,DAKAR,Rufisque,2015/2016,Cowpea,858.0,397,0.462704,130000,0.660000,NaN,NaN


## Cleaning up the data

In [6]:
drop_cols = ['Exact locations?', 'data from http://www.maer.gouv.sn/focus/statistiques-cles/']

In [7]:
crop_yield.drop(columns=drop_cols, inplace=True)

In [8]:
crop_yield.Department = crop_yield.Department.replace('oussouye', 'Oussouye') 
crop_yield.Department = crop_yield.Department.replace('DAGANA', 'Dagana') 
crop_yield.Department = crop_yield.Department.replace('GUINGUINEO', 'Guinguinéo') 
crop_yield.Department = crop_yield.Department.replace('KAOLACK', 'Kaolack') 
crop_yield.Department = crop_yield.Department.replace('NIORO', 'Nioro du Rip') 
crop_yield.Department = crop_yield.Department.replace('PODOR', 'Podor') 
crop_yield.Department = crop_yield.Department.replace('SAINT-LOUIS', 'Saint-Louis') 
crop_yield.Department = crop_yield.Department.replace('SARAYA', 'Saraya') 
crop_yield.Department = crop_yield.Department.replace('SEDHIOU', 'Sédhiou') 
crop_yield.Department = crop_yield.Department.replace('THIES', 'Thiès') 
crop_yield.Department = crop_yield.Department.replace('Koumpentoum', 'Koupentoum') # correct typo

In [9]:
crop_yield.Region.unique()

array(['DAKAR', 'FATICK', 'DIOURBEL', 'KAFFRINE', 'KAOLACK', 'KEDOUGOU',
       'KOLDA', 'LOUGA', 'MATAM', 'SAINT-LOUIS', 'SEDHIOU', 'TAMBACOUNDA',
       'THIES', 'ZIGUINCHOR'], dtype=object)

In [10]:
regions_yield = sorted(list(crop_yield.Region.unique()))
#regions_yield

In [11]:
departments_yield = sorted(list(crop_yield.Department.unique()))
departments_yield.append('Dakar')
departments_yield.append('Guédiawaye')
departments_yield.append('Pikine')

departments_yield = sorted(departments_yield)
departments_yield, len(departments_yield)

(['Bakel',
  'Bambey',
  'Bignona',
  'Birkelane',
  'Bounkiling',
  'Dagana',
  'Dakar',
  'Diourbel',
  'Fatick',
  'Foundiougne',
  'Gossas',
  'Goudiry',
  'Goudomp',
  'Guinguinéo',
  'Guédiawaye',
  'Kaffrine',
  'Kanel',
  'Kaolack',
  'Kolda',
  'Koungheul',
  'Koupentoum',
  'Kébémer',
  'Kédougou',
  'Linguère',
  'Louga',
  'Malem Hodar',
  'Matam',
  'Mbacké',
  'Mbour',
  'Médina Yoro Foulah',
  'Nioro du Rip',
  'Oussouye',
  'Pikine',
  'Podor',
  'Ranérou',
  'Rufisque',
  'Saint-Louis',
  'Salémata',
  'Saraya',
  'Sédhiou',
  'Tamba',
  'Thiès',
  'Tivaooune',
  'Vélingara',
  'Ziguinchor'],
 45)

In [82]:
Senegal_df = gpd.read_file(str(shape_file_path))

In [83]:
Senegal_df.head()

,GID_0,NAME_0,GID_1,NAME_1,NL_NAME_1,GID_2,NAME_2,VARNAME_2,NL_NAME_2,TYPE_2,ENGTYPE_2,CC_2,HASC_2,geometry
0,SEN,Senegal,SEN.1_1,Dakar,None,SEN.1.1_1,Dakar,None,None,Département,Department,11,SN.DK.DK,"MULTIPOLYGON (((-17.46736 14.65097, -17.46736 ..."
1,SEN,Senegal,SEN.1_1,Dakar,None,SEN.1.2_1,Guédiawaye,None,None,Département,Department,14,SN.DK.GD,"POLYGON ((-17.40012 14.75731, -17.40051 14.757..."
2,SEN,Senegal,SEN.1_1,Dakar,None,SEN.1.3_1,Pikine,None,None,Département,Department,12,SN.DK.PK,"POLYGON ((-17.39897 14.73781, -17.39903 14.737..."
3,SEN,Senegal,SEN.1_1,Dakar,None,SEN.1.4_1,Rufisque,Rufisque-Bargny,None,Département,Department,13,SN.DK.RF,"POLYGON ((-17.16949 14.63201, -17.16952 14.632..."
4,SEN,Senegal,SEN.2_1,Diourbel,None,SEN.2.1_1,Bambey,None,None,Département,Department,31,SN.DB.BB,"POLYGON ((-16.37093 14.60763, -16.37140 14.607..."


In [84]:
regions_geo = sorted(list(Senegal_df.NAME_1.unique()))
#regions_geo

In [85]:
department_geo = sorted(list(Senegal_df.NAME_2.unique()))
department_geo, len(department_geo)

(['Bakel',
  'Bambey',
  'Bignona',
  'Birkilane',
  'Bounkiling',
  'Dagana',
  'Dakar',
  'Diourbel',
  'Fatick',
  'Foundiougne',
  'Gossas',
  'Goudiry',
  'Goudomp',
  'Guinguinéo',
  'Guédiawaye',
  'Kaffrine',
  'Kanel',
  'Kaolack',
  'Kolda',
  'Koungheul',
  'Koupentoum',
  'Kébémer',
  'Kédougou',
  'Linguère',
  'Louga',
  'Malème Hodar',
  'Matam',
  'Mbacké',
  'Mbour',
  'Médina Yoro Foula',
  'Nioro du Rip',
  'Oussouye',
  'Pikine',
  'Podor',
  'Ranérou Ferlo',
  'Rufisque',
  'Saint-Louis',
  'Salémata',
  'Saraya',
  'Sédhiou',
  'Tambacounda',
  'Thiès',
  'Tivaouane',
  'Vélingara',
  'Ziguinchor'],
 45)

In [86]:
zip_iterator = zip(regions_yield, regions_geo)
region_mapping = dict(zip_iterator)
region_mapping

{'DAKAR': 'Dakar',
 'DIOURBEL': 'Diourbel',
 'FATICK': 'Fatick',
 'KAFFRINE': 'Kaffrine',
 'KAOLACK': 'Kaolack',
 'KEDOUGOU': 'Kolda',
 'KOLDA': 'Kédougou',
 'LOUGA': 'Louga',
 'MATAM': 'Matam',
 'SAINT-LOUIS': 'Saint-Louis',
 'SEDHIOU': 'Sédhiou',
 'TAMBACOUNDA': 'Tambacounda',
 'THIES': 'Thiès',
 'ZIGUINCHOR': 'Ziguinchor'}

In [87]:
# this mapping is messed up due to alphabetic sorting with and without accents!
# cleaning up the mis-matched items
region_mapping['KEDOUGOU'] = 'Kédougou'
region_mapping['KOLDA'] = 'Kolda'
region_mapping

{'DAKAR': 'Dakar',
 'DIOURBEL': 'Diourbel',
 'FATICK': 'Fatick',
 'KAFFRINE': 'Kaffrine',
 'KAOLACK': 'Kaolack',
 'KEDOUGOU': 'Kédougou',
 'KOLDA': 'Kolda',
 'LOUGA': 'Louga',
 'MATAM': 'Matam',
 'SAINT-LOUIS': 'Saint-Louis',
 'SEDHIOU': 'Sédhiou',
 'TAMBACOUNDA': 'Tambacounda',
 'THIES': 'Thiès',
 'ZIGUINCHOR': 'Ziguinchor'}

In [88]:
zip_iterator = zip(departments_yield, department_geo)
departments_mapping = dict(zip_iterator)
departments_mapping

{'Bakel': 'Bakel',
 'Bambey': 'Bambey',
 'Bignona': 'Bignona',
 'Birkelane': 'Birkilane',
 'Bounkiling': 'Bounkiling',
 'Dagana': 'Dagana',
 'Dakar': 'Dakar',
 'Diourbel': 'Diourbel',
 'Fatick': 'Fatick',
 'Foundiougne': 'Foundiougne',
 'Gossas': 'Gossas',
 'Goudiry': 'Goudiry',
 'Goudomp': 'Goudomp',
 'Guinguinéo': 'Guinguinéo',
 'Guédiawaye': 'Guédiawaye',
 'Kaffrine': 'Kaffrine',
 'Kanel': 'Kanel',
 'Kaolack': 'Kaolack',
 'Kolda': 'Kolda',
 'Koungheul': 'Koungheul',
 'Koupentoum': 'Koupentoum',
 'Kébémer': 'Kébémer',
 'Kédougou': 'Kédougou',
 'Linguère': 'Linguère',
 'Louga': 'Louga',
 'Malem Hodar': 'Malème Hodar',
 'Matam': 'Matam',
 'Mbacké': 'Mbacké',
 'Mbour': 'Mbour',
 'Médina Yoro Foulah': 'Médina Yoro Foula',
 'Nioro du Rip': 'Nioro du Rip',
 'Oussouye': 'Oussouye',
 'Pikine': 'Pikine',
 'Podor': 'Podor',
 'Ranérou': 'Ranérou Ferlo',
 'Rufisque': 'Rufisque',
 'Saint-Louis': 'Saint-Louis',
 'Salémata': 'Salémata',
 'Saraya': 'Saraya',
 'Sédhiou': 'Sédhiou',
 'Tamba': 'Tambaco

In [89]:
#crop_yield = crop_yield.replace({"Department" : departments_mapping}) # does not work
#crop_yield = crop_yield.replace({"Region" : region_mapping}) # does not work
crop_yield.Department = crop_yield.Department.replace(departments_mapping)
crop_yield.Region = crop_yield.Region.replace(region_mapping)
crop_yield

,Region,Department,Year,Crop,Area planted (ha),Production (T),Yield (ton/ha),Total surface area department (ha),percentage of area planted in region for crop type
0,Dakar,Rufisque,2014/2015,Arachide,2943.0,589,0.200136,130000,2.263846
1,Dakar,Rufisque,2015/2016,Arachide,1233.0,935,0.758313,130000,0.948462
2,Dakar,Rufisque,2016/2017,Arachide,1248.0,257,0.205929,130000,0.960000
3,Dakar,Rufisque,2014/2015,Cowpea,1178.0,177,0.150255,130000,0.906154
4,Dakar,Rufisque,2015/2016,Cowpea,858.0,397,0.462704,130000,0.660000
...,...,...,...,...,...,...,...,...,...
243,Ziguinchor,Bignona,2015,Rice,16900.0,39884,2.360000,529500,3.191690
244,Ziguinchor,Bignona,2016,Rice,16900.0,25350,1.500000,529500,3.191690
245,Ziguinchor,Oussouye,2014,Rice,4309.0,6818,1.582270,89100,4.836139
246,Ziguinchor,Oussouye,2015,Rice,14500.0,26999,1.862000,89100,16.273850


In [90]:
crop_yield.groupby(['Department', 'Year']).sum()

Area planted (ha)  Yield (ton/ha)  \
Department Year                                      
Bakel      2014             1679.0        7.154994   
           2015             2841.0        7.970914   
           2016             2721.0        7.107287   
Bambey     2014            25000.0        0.500000   
           2015            27900.0        0.800681   
...                            ...             ...   
Vélingara  2015            37895.0        3.565566   
           2016            35670.0        3.047023   
Ziguinchor 2014             2720.0        3.210202   
           2015             6826.0        2.882912   
           2016             7578.0        0.949937   

                 percentage of area planted in region for crop type  
Department Year                                                      
Bakel      2014                                           0.266932   
           2015                                           0.451669   
           2016                                           0.432591   
Bambey     2014                                          18.504811   
           2015                                          20.651369   
...                                                            ...   
Vélingara  2015                                           6.973684   
           2016                                           6.564225   
Ziguinchor 2014                                           2.359063   
           2015                                           5.920208   
           2016                                           6.572420   

[120 rows x 3 columns]

## Join both datasets

In [91]:
yield_gpd = pd.merge(crop_yield, Senegal_df,  how='left', left_on=['Region','Department'], right_on = ['NAME_1','NAME_2'])

In [92]:
yield_gpd.head()

,Region,Department,Year,Crop,Area planted (ha),Production (T),Yield (ton/ha),Total surface area department (ha),percentage of area planted in region for crop type,GID_0,...,NL_NAME_1,GID_2,NAME_2,VARNAME_2,NL_NAME_2,TYPE_2,ENGTYPE_2,CC_2,HASC_2,geometry
0,Dakar,Rufisque,2014/2015,Arachide,2943.0,589,0.200136,130000,2.263846,SEN,...,None,SEN.1.4_1,Rufisque,Rufisque-Bargny,None,Département,Department,13,SN.DK.RF,"POLYGON ((-17.16949 14.63201, -17.16952 14.632..."
1,Dakar,Rufisque,2015/2016,Arachide,1233.0,935,0.758313,130000,0.948462,SEN,...,None,SEN.1.4_1,Rufisque,Rufisque-Bargny,None,Département,Department,13,SN.DK.RF,"POLYGON ((-17.16949 14.63201, -17.16952 14.632..."
2,Dakar,Rufisque,2016/2017,Arachide,1248.0,257,0.205929,130000,0.960000,SEN,...,None,SEN.1.4_1,Rufisque,Rufisque-Bargny,None,Département,Department,13,SN.DK.RF,"POLYGON ((-17.16949 14.63201, -17.16952 14.632..."
3,Dakar,Rufisque,2014/2015,Cowpea,1178.0,177,0.150255,130000,0.906154,SEN,...,None,SEN.1.4_1,Rufisque,Rufisque-Bargny,None,Département,Department,13,SN.DK.RF,"POLYGON ((-17.16949 14.63201, -17.16952 14.632..."
4,Dakar,Rufisque,2015/2016,Cowpea,858.0,397,0.462704,130000,0.660000,SEN,...,None,SEN.1.4_1,Rufisque,Rufisque-Bargny,None,Département,Department,13,SN.DK.RF,"POLYGON ((-17.16949 14.63201, -17.16952 14.632..."


In [93]:
type(yield_gpd)

pandas.core.frame.DataFrame

In [94]:
# convert joined dataframe to geopandas
yield_gpd = gpd.GeoDataFrame(yield_gpd, geometry='geometry')

In [95]:
type(yield_gpd)

geopandas.geodataframe.GeoDataFrame

## Feature Engineering

In [96]:
yield_gpd.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [97]:
sqm_to_ha = 0.0001

In [98]:
# Project to crs that uses meters as distance measure
yield_3395 = yield_gpd.to_crs('epsg:3395')
yield_3395['department_area_ha'] = yield_3395.geometry.area *sqm_to_ha
yield_gpd = yield_3395.to_crs('epsg:4326')

In [99]:
# Area of Departement Rufisque: 371,8 km2
ha_to_km2 = 0.01
yield_gpd['department_area_km2']= yield_gpd['department_area_ha'] * ha_to_km2

In [100]:
yield_gpd

,Region,Department,Year,Crop,Area planted (ha),Production (T),Yield (ton/ha),Total surface area department (ha),percentage of area planted in region for crop type,GID_0,...,NAME_2,VARNAME_2,NL_NAME_2,TYPE_2,ENGTYPE_2,CC_2,HASC_2,geometry,department_area_ha,department_area_km2
0,Dakar,Rufisque,2014/2015,Arachide,2943.0,589,0.200136,130000,2.263846,SEN,...,Rufisque,Rufisque-Bargny,None,Département,Department,13,SN.DK.RF,"POLYGON ((-17.16949 14.63201, -17.16952 14.632...",40817.799669,408.177997
1,Dakar,Rufisque,2015/2016,Arachide,1233.0,935,0.758313,130000,0.948462,SEN,...,Rufisque,Rufisque-Bargny,None,Département,Department,13,SN.DK.RF,"POLYGON ((-17.16949 14.63201, -17.16952 14.632...",40817.799669,408.177997
2,Dakar,Rufisque,2016/2017,Arachide,1248.0,257,0.205929,130000,0.960000,SEN,...,Rufisque,Rufisque-Bargny,None,Département,Department,13,SN.DK.RF,"POLYGON ((-17.16949 14.63201, -17.16952 14.632...",40817.799669,408.177997
3,Dakar,Rufisque,2014/2015,Cowpea,1178.0,177,0.150255,130000,0.906154,SEN,...,Rufisque,Rufisque-Bargny,None,Département,Department,13,SN.DK.RF,"POLYGON ((-17.16949 14.63201, -17.16952 14.632...",40817.799669,408.177997
4,Dakar,Rufisque,2015/2016,Cowpea,858.0,397,0.462704,130000,0.660000,SEN,...,Rufisque,Rufisque-Bargny,None,Département,Department,13,SN.DK.RF,"POLYGON ((-17.16949 14.63201, -17.16952 14.632...",40817.799669,408.177997
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
243,Ziguinchor,Bignona,2015,Rice,16900.0,39884,2.360000,529500,3.191690,SEN,...,Bignona,None,None,Département,Department,21,SN.ZG.BG,"POLYGON ((-16.33970 12.59094, -16.34010 12.590...",562048.113548,5620.481135
244,Ziguinchor,Bignona,2016,Rice,16900.0,25350,1.500000,529500,3.191690,SEN,...,Bignona,None,None,Département,Department,21,SN.ZG.BG,"POLYGON ((-16.33970 12.59094, -16.34010 12.590...",562048.113548,5620.481135
245,Ziguinchor,Oussouye,2014,Rice,4309.0,6818,1.582270,89100,4.836139,SEN,...,Oussouye,None,None,Département,Department,22,SN.ZG.OU,"POLYGON ((-16.45373 12.43734, -16.45239 12.436...",91944.771922,919.447719
246,Ziguinchor,Oussouye,2015,Rice,14500.0,26999,1.862000,89100,16.273850,SEN,...,Oussouye,None,None,Département,Department,22,SN.ZG.OU,"POLYGON ((-16.45373 12.43734, -16.45239 12.436...",91944.771922,919.447719


In [101]:
crop_area = yield_gpd.groupby(['Department', 'Year']).sum()
crop_area

Area planted (ha)  Yield (ton/ha)  \
Department Year                                      
Bakel      2014             1679.0        7.154994   
           2015             2841.0        7.970914   
           2016             2721.0        7.107287   
Bambey     2014            25000.0        0.500000   
           2015            27900.0        0.800681   
...                            ...             ...   
Vélingara  2015            37895.0        3.565566   
           2016            35670.0        3.047023   
Ziguinchor 2014             2720.0        3.210202   
           2015             6826.0        2.882912   
           2016             7578.0        0.949937   

                 percentage of area planted in region for crop type  \
Department Year                                                       
Bakel      2014                                           0.266932    
           2015                                           0.451669    
           2016                                           0.432591    
Bambey     2014                                          18.504811    
           2015                                          20.651369    
...                                                            ...    
Vélingara  2015                                           6.973684    
           2016                                           6.564225    
Ziguinchor 2014                                           2.359063    
           2015                                           5.920208    
           2016                                           6.572420    

                 department_area_ha  department_area_km2  
Department Year                                           
Bakel      2014        1.452544e+06         14525.443765  
           2015        1.452544e+06         14525.443765  
           2016        1.452544e+06         14525.443765  
Bambey     2014        1.447557e+05          1447.556769  
           2015        1.447557e+05          1447.556769  
...                             ...                  ...  
Vélingara  2015        1.157917e+06         11579.165881  
           2016        1.157917e+06         11579.165881  
Ziguinchor 2014        2.382932e+05          2382.932135  
           2015        2.382932e+05          2382.932135  
           2016        2.382932e+05          2382.932135  

[120 rows x 5 columns]

In [102]:
crop_area['area_yearly_diff']= crop_area.groupby('Department')['Area planted (ha)'].diff().fillna(0)
crop_area['area_yearly_diff_pct']= crop_area.groupby('Department')['Area planted (ha)'].pct_change().fillna(0)
#df.pct_change()
crop_area

Area planted (ha)  Yield (ton/ha)  \
Department Year                                      
Bakel      2014             1679.0        7.154994   
           2015             2841.0        7.970914   
           2016             2721.0        7.107287   
Bambey     2014            25000.0        0.500000   
           2015            27900.0        0.800681   
...                            ...             ...   
Vélingara  2015            37895.0        3.565566   
           2016            35670.0        3.047023   
Ziguinchor 2014             2720.0        3.210202   
           2015             6826.0        2.882912   
           2016             7578.0        0.949937   

                 percentage of area planted in region for crop type  \
Department Year                                                       
Bakel      2014                                           0.266932    
           2015                                           0.451669    
           2016                                           0.432591    
Bambey     2014                                          18.504811    
           2015                                          20.651369    
...                                                            ...    
Vélingara  2015                                           6.973684    
           2016                                           6.564225    
Ziguinchor 2014                                           2.359063    
           2015                                           5.920208    
           2016                                           6.572420    

                 department_area_ha  department_area_km2  area_yearly_diff  \
Department Year                                                              
Bakel      2014        1.452544e+06         14525.443765               0.0   
           2015        1.452544e+06         14525.443765            1162.0   
           2016        1.452544e+06         14525.443765            -120.0   
Bambey     2014        1.447557e+05          1447.556769               0.0   
           2015        1.447557e+05          1447.556769            2900.0   
...                             ...                  ...               ...   
Vélingara  2015        1.157917e+06         11579.165881           18195.0   
           2016        1.157917e+06         11579.165881           -2225.0   
Ziguinchor 2014        2.382932e+05          2382.932135               0.0   
           2015        2.382932e+05          2382.932135            4106.0   
           2016        2.382932e+05          2382.932135             752.0   

                 area_yearly_diff_pct  
Department Year                        
Bakel      2014              0.000000  
           2015              0.692079  
           2016             -0.042239  
Bambey     2014              0.000000  
           2015              0.116000  
...                               ...  
Vélingara  2015              0.923604  
           2016             -0.058715  
Ziguinchor 2014              0.000000  
           2015              1.509559  
           2016              0.110167  

[120 rows x 7 columns]

## reorganizing the column structure of the dataframe

In [103]:
# credits for this procedure to : https://stackoverflow.com/questions/50660949/moving-a-dataframe-column-and-changing-column-order
cols = crop_area.columns.tolist() # list the columns in the df
cols.insert(1, cols.pop(cols.index('area_yearly_diff'))) # Assign new position for the specified column 
cols.insert(2, cols.pop(cols.index('area_yearly_diff_pct')))
crop_area = crop_area.reindex(columns= cols) # recreate the new dataframe in the changed order

In [104]:
crop_area

Area planted (ha)  area_yearly_diff  area_yearly_diff_pct  \
Department Year                                                              
Bakel      2014             1679.0               0.0              0.000000   
           2015             2841.0            1162.0              0.692079   
           2016             2721.0            -120.0             -0.042239   
Bambey     2014            25000.0               0.0              0.000000   
           2015            27900.0            2900.0              0.116000   
...                            ...               ...                   ...   
Vélingara  2015            37895.0           18195.0              0.923604   
           2016            35670.0           -2225.0             -0.058715   
Ziguinchor 2014             2720.0               0.0              0.000000   
           2015             6826.0            4106.0              1.509559   
           2016             7578.0             752.0              0.110167   

                 Yield (ton/ha)  \
Department Year                   
Bakel      2014        7.154994   
           2015        7.970914   
           2016        7.107287   
Bambey     2014        0.500000   
           2015        0.800681   
...                         ...   
Vélingara  2015        3.565566   
           2016        3.047023   
Ziguinchor 2014        3.210202   
           2015        2.882912   
           2016        0.949937   

                 percentage of area planted in region for crop type  \
Department Year                                                       
Bakel      2014                                           0.266932    
           2015                                           0.451669    
           2016                                           0.432591    
Bambey     2014                                          18.504811    
           2015                                          20.651369    
...                                                            ...    
Vélingara  2015                                           6.973684    
           2016                                           6.564225    
Ziguinchor 2014                                           2.359063    
           2015                                           5.920208    
           2016                                           6.572420    

                 department_area_ha  department_area_km2  
Department Year                                           
Bakel      2014        1.452544e+06         14525.443765  
           2015        1.452544e+06         14525.443765  
           2016        1.452544e+06         14525.443765  
Bambey     2014        1.447557e+05          1447.556769  
           2015        1.447557e+05          1447.556769  
...                             ...                  ...  
Vélingara  2015        1.157917e+06         11579.165881  
           2016        1.157917e+06         11579.165881  
Ziguinchor 2014        2.382932e+05          2382.932135  
           2015        2.382932e+05          2382.932135  
           2016        2.382932e+05          2382.932135  

[120 rows x 7 columns]